In [3]:
import folium
import pandas as pd

In [4]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [5]:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df=pd.read_csv(URL)

In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


### Mark all launch sites on a map

In [7]:
# Launch site coordinates
launch_sites = [
    ("CCAFS LC-40", 28.56230197, -80.57735648, "CCAFS LC-40"),
    ("CCAFS SLC-40", 28.56319718, -80.57682003, "CCAFS SLC-40"),
    ("KSC LC-39A", 28.57325457, -80.64689529, "KSC LC-39A"),
    ("VAFB SLC-4E", 34.63283416, -120.6107455, "VAFB SLC-4E")
]

# Start map centered roughly between Florida and California
site_map = folium.Map(location=[30, -95], zoom_start=4)

# Add circles and markers for each site
for name, lat, lon, label in launch_sites:
    # Circle with popup
    circle = folium.Circle(
        location=[lat, lon],
        radius=1000,
        color='#d35400',
        fill=True
    ).add_child(folium.Popup(name))
    
    # Text label marker
    marker = folium.Marker(
        location=[lat, lon],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{label}</b></div>'
        )
    )
    
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map


### Mark the success/failed launches for each site on the map


In [8]:


# Create marker color column based on 'class'
# class = 1 → success (green), class = 0 → failure (red)
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Create base map centered roughly between Florida and California
site_map = folium.Map(location=[30, -95], zoom_start=4)

# Initialize marker cluster
marker_cluster = MarkerCluster().add_to(site_map)

# Add markers for each launch record
for idx, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],  # replace with your column names for coordinates
        popup=f"Launch Site: {row['Launch Site']}<br>Outcome: {'Success' if row['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=row['marker_color'])
    ).add_to(marker_cluster)

# Display map
site_map


### Calculate the distances between a launch site to its proximities

In [9]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [10]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [11]:
from folium import PolyLine

# let's use CCAFS SLC-40
launch_site_lat = 28.56319718
launch_site_lon = -80.57682003

# Closest coastline coordinates
coastline_lat = 28.56145
coastline_lon = -80.5677

# Calculate the distance
distance_coastline = calculate_distance(
    launch_site_lat, launch_site_lon,
    coastline_lat, coastline_lon
)

# Add marker for coastline
coast_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    popup="Closest Coastline Point",
    icon=folium.Icon(color="blue", icon="info-sign")
)
site_map.add_child(coast_marker)

# Add a distance label marker midway between the two points
midpoint = [
    (launch_site_lat + coastline_lat) / 2,
    (launch_site_lon + coastline_lon) / 2
]
distance_marker = folium.Marker(
    midpoint,
    icon=DivIcon(
        icon_size=(150, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline)
    )
)
site_map.add_child(distance_marker)

# Draw a line between the launch site and coastline
line = PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    weight=2,
    color='red'
)
site_map.add_child(line)

site_map
